In [17]:
# load the file
import sys
import io
import os

fileDir = "../Data/canopies_clean/"
listfiles = os.listdir(fileDir)
allLabeledPapersInFile_original = []

for file in listfiles:
    if not file.startswith('.'):
        with open(fileDir+file, 'r', encoding = 'utf8') as f:
            for line in f:
                read_data = line.split("\t")
                # get ride of bad formated lines
                if(len(read_data)==13):
                    paper_detail = {"paperID": read_data[0], "authorID":read_data[1],
                                    "author_position":read_data[2], "total_author": read_data[3],
                                    "author_name": read_data[4], "co_authors": read_data[5], 
                                    "department": read_data[6], "vendor": read_data[7], 
                                    "mesh": read_data[8], "keywords": read_data[9], 
                                    "publish_year": read_data[10], "citation graph": read_data[11]}
                    allLabeledPapersInFile_original.append(paper_detail)
        f.close()
print("Total labeled records:",len(allLabeledPapersInFile_original), "records")

Total use records: 140266 records


In [18]:
# data column
for key in allLabeledPapersInFile_original[0]:
    print(key)

paperID
authorID
author_position
total_author
author_name
co_authors
department
vendor
mesh
keywords
publish_year
citation graph


In [19]:
# merge author name based on their id, replace abstract name to full name if full name available
# this case, we assume that author id is unique identifier means each author will only have one author ID
import pandas as pd
allLabelPapersInFile_dataframe = pd.DataFrame(allLabeledPapersInFile_original)
print(list(allLabelPapersInFile_dataframe.columns.values))
print(allLabelPapersInFile_dataframe[:5])
print(len(allLabelPapersInFile_dataframe.authorID.unique()))

['authorID', 'author_name', 'author_position', 'citation graph', 'co_authors', 'department', 'keywords', 'mesh', 'paperID', 'publish_year', 'total_author', 'vendor']
              authorID      author_name author_position  \
0  0000-0002-9697-0962  jonathan m read               4   
1  0000-0002-9697-0962         j m read               1   
2  0000-0002-9697-0962    jonathan read               1   
3  0000-0002-9697-0962  jonathan m read               3   
4  0000-0002-9697-0962  jonathan m read               7   

                                      citation graph  \
0  10.3345/kjp.2012.55.12.474$$10.1016/j.vaccine....   
1  10.3201/eid1601.090401$$10.1016/j.jcv.2008.12....   
2  10.3201/eid1702.100458$$10.1093/aje/kwq243$$10...   
3  10.1097/inf.0b013e3181fefc51$$10.1016/s1473-30...   
4  10.1097/inf.0b013e3181ecbf7e$$10.1093/aje/kwt2...   

                                          co_authors  \
0  truelove zhu lessler riley wang kwok guan jian...   
1  hungerford cooke vivancos i

In [20]:
from collections import Counter

# create author ID to name map
author_ID_to_name_map = []

for paper in allLabeledPapersInFile_original:
    if(paper["authorID"] not in [item["author_ID"] for item in author_ID_to_name_map]):
        author_ID_to_name_map.append({"author_name": paper["author_name"], "author_ID": paper["authorID"]})

print(len(author_ID_to_name_map))
# extract author with same name
# author with same name different author ID
sameNameAuthor = []
for i in author_ID_to_name_map:
    for j in author_ID_to_name_map:
        if(j["author_name"]==i["author_name"] and j["author_ID"]!=i["author_ID"]):
            if(i["author_ID"] not in [item["author_ID"] for item in sameNameAuthor]):
                sameNameAuthor.append(i)
            if(j["author_ID"] not in [item["author_ID"] for item in sameNameAuthor]):
                sameNameAuthor.append(j)

# # count author with same name
# sameNameCounter = Counter([author["author_name"] for author in sameNameAuthor])
# for key, value in sameNameCounter.items():
#     if value>=2:
#         print(value, "people have same name:", key)

# for author in sameNameAuthor:
#     print("Author: {k}, Author ID: {c}".format(k=author["author_name"], c=author["author_ID"]))    

9914


In [8]:
# assume author ID are unique name identifier
from collections import Counter
# find the paper that each author write
c = Counter([paper["authorID"] for paper in allLabeledPapersInFile_original])
#for key, value in c.items():
c.most_common(3) 

[('0000-0002-2381-2349', 587),
 ('0000-0002-9955-6003', 487),
 ('0000-0002-4295-6129', 423)]

In [6]:
# filter out authors that have same name but write less paper than threshold

# set up threshold
threshold = 30

print(len(allLabeledPapersInFile_original))
# count number of paper author write
c = Counter([paper["authorID"] for paper in allLabeledPapersInFile_original])
for key, value in c.items():
    for author in sameNameAuthor:
        if(key == author["author_ID"]):
            author["paper_count"] = value

sameNameAuthorWithCount = sorted(sameNameAuthor, key=lambda k: (k['author_name'], k["paper_count"]))
# collect authors that have repeated name and write more paper than threshold
uniqueAuthors = []
temp = []
authorFilter = []
for author in sameNameAuthorWithCount:
    if(author["paper_count"]>threshold):
        temp.append(author)
        #print("Author name: {n}, Author: {k}, Paper count: {c}".format(n=author["author_name"],k=author["author_ID"], c= author["paper_count"]))

for author in temp:
    if(author["author_name"] not in uniqueAuthors):
        uniqueAuthors.append(author["author_name"])
    else: 
        if author["author_name"] not in authorFilter:
            authorFilter.append(author["author_name"])
# collect filtered data
filteredAuthors = []
for author in temp:
    if(author["author_name"] in authorFilter):
        filteredAuthors.append(author)

for author in filteredAuthors:
    print("Author name: {n}, Author: {k}, Paper count: {c}".format(n=author["author_name"],k=author["author_ID"], c= author["paper_count"]))


4163770
Author name: chung-may yang, Author: 0000-0003-4082-420X, Paper count: 42
Author name: chung-may yang, Author: 0000-0002-4328-8716, Paper count: 71
Author name: david g lloyd, Author: 0000-0003-0658-8995, Paper count: 50
Author name: david g lloyd, Author: 0000-0002-0824-9682, Paper count: 104
Author name: francisco ortega, Author: 0000-0002-4730-9270, Paper count: 38
Author name: francisco ortega, Author: 0000-0003-2111-769X, Paper count: 86
Author name: lei wang, Author: 0000-0002-5859-2526, Paper count: 53
Author name: lei wang, Author: 0000-0003-3870-3388, Paper count: 64
Author name: michael moore, Author: 0000-0003-3074-6631, Paper count: 38
Author name: michael moore, Author: 0000-0002-5127-4509, Paper count: 45
Author name: michael wagner, Author: 0000-0003-2589-6440, Paper count: 98
Author name: michael wagner, Author: 0000-0002-9778-7684, Paper count: 141
